# Industrial Equipment Sensors – Data Cleaning Workflow

This notebook documents the complete cleaning workflow for the **Equipment Sensor Dataset**.

It assumes you have a raw file named `equipment_sensors_raw.xlsx` in the same folder.

Main steps:
- Load dataset
- Standardize column names
- Clean string-based fields (status, operator, location, shift)
- Parse mixed timestamp formats
- Convert noisy numeric sensor readings
- Handle missing data
- Remove duplicates
- Treat extreme sensor outliers
- Export cleaned dataset as `equipment_sensors_clean.xlsx`


In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 160)
print('pandas version:', pd.__version__)

## 1. Load raw data

In [ ]:
raw_path = 'equipment_sensors_raw.xlsx'
df_raw = pd.read_excel(raw_path)
df_raw.head()

In [ ]:
df_raw.info()

## 2. Standardize column names

In [ ]:
df = df_raw.copy()
df.columns = (
    df.columns.str.strip()
              .str.lower()
              .str.replace(r'\s+', '_', regex=True)
              .str.replace('[()]', '', regex=True)
)
df.head()

## 3. Clean string-based categorical fields

In [ ]:
# Clean object columns
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype(str).str.strip()
    df[col] = df[col].replace({'': np.nan, 'na': np.nan, 'NA': np.nan, '?': np.nan})

cat_cols = ['location', 'status', 'operator', 'shift']
for col in cat_cols:
    if col in df.columns:
        df[col] = df[col].str.upper()

df[cat_cols].head()

## 4. Convert timestamps

In [ ]:
if 'timestamp' in df.columns:
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df[['timestamp']].head()

## 5. Convert noisy numeric sensor readings

In [ ]:
numeric_cols = ['temperature_c', 'pressure_bar', 'vibration_mms']

for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.replace(r'[^0-9\.-]', '', regex=True)
        df[col] = pd.to_numeric(df[col], errors='coerce')

df[numeric_cols].head()

## 6. Handle missing values

In [ ]:
# Drop rows missing critical ID
if 'record_id' in df.columns:
    df = df.dropna(subset=['record_id'])

# Impute numeric columns
for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())

df.isna().sum()

## 7. Remove duplicates

In [ ]:
before = len(df)
df = df.drop_duplicates()
after = len(df)
print('Duplicates removed:', before - after)

## 8. Outlier treatment (temperature, pressure, vibration)

In [ ]:
def cap_outliers(col):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    upper = q3 + 1.5 * iqr
    return df[col].clip(upper=upper)

for col in numeric_cols:
    if col in df.columns:
        df[col + '_capped'] = cap_outliers(col)

df[[c for c in df.columns if 'temperature' in c]].head()

## 9. Final checks

In [ ]:
df.info()

In [ ]:
df.describe(include='all').transpose().head(20)

## 10. Export cleaned dataset

In [ ]:
clean_path = 'equipment_sensors_clean.xlsx'
df.to_excel(clean_path, index=False)
print('Cleaned file saved to:', clean_path)

---
### Notes for Upwork / Portfolio
- Demonstrates cleaning industrial sensor logs
- Handles mixed timestamp formats
- Converts noisy sensor readings
- Removes outliers, missing data, duplicates
- Produces ML-ready dataset for anomaly detection and predictive maintenance